In [2]:
pip install pandas transformers datasets torch


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from transformers import RobertaTokenizer


/Users/hannahdestiny/Desktop/FakeProfileDetection/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/hannahdestiny/Desktop/FakeProfileDetection/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <modul

In [ ]:
training_df = pd.read_csv("/Users/hannahdestiny/Desktop/FakeProfileDetection/dev/csv/structured_train.csv")
testing_df = pd.read_csv("/Users/hannahdestiny/Desktop/FakeProfileDetection/dev/csv/structured_test.csv")
pd.set_option('display.max_colwidth', None)
training_df.head(50)

In [ ]:
from CleanDatasets import CleanDatasets

columns_to_rename_dict = {"screen_name": "username", "account.type": "is_human"}
columns_to_drop_list = ["class_type"]
column_to_binary = "account.type"


training_cleaner = CleanDatasets(training_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_training_df = training_cleaner.clean_df()
testing_cleaner = CleanDatasets(testing_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_testing_df = testing_cleaner.clean_df()
clean_training_df.head(50)


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)
